<a href="https://colab.research.google.com/github/omkarpat/eeg-mood-classification/blob/master/Models/Baseline_Raw_Spectrogram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/CSE\ 240/Project/Data/spectrogram_raw/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/My Drive/CSE 240/Project/Data/spectrogram_raw/'
/content/gdrive/My Drive/CSE 240/Project/Data/spectrogram_images


In [2]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold

%tensorflow_version 2.x
import tensorflow
print(tensorflow.__version__)
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate, SeparableConv2D, MaxPooling3D, Conv3D, DepthwiseConv2D
from tensorflow.keras.models import model_from_json

TensorFlow 2.x selected.
2.1.0


In [5]:
!ls

S001E01.npy  S006E01.npy  S011E01.npy  S016E01.npy  S021E01.npy  S026E01.npy
S001E02.npy  S006E02.npy  S011E02.npy  S016E02.npy  S021E02.npy  S026E02.npy
S001E03.npy  S006E03.npy  S011E03.npy  S016E03.npy  S021E03.npy  S026E03.npy
S001E04.npy  S006E04.npy  S011E04.npy  S016E04.npy  S021E04.npy  S026E04.npy
S002E01.npy  S007E01.npy  S012E01.npy  S017E01.npy  S022E01.npy  S027E01.npy
S002E02.npy  S007E02.npy  S012E02.npy  S017E02.npy  S022E02.npy  S027E02.npy
S002E03.npy  S007E03.npy  S012E03.npy  S017E03.npy  S022E03.npy  S027E03.npy
S002E04.npy  S007E04.npy  S012E04.npy  S017E04.npy  S022E04.npy  S027E04.npy
S003E01.npy  S008E01.npy  S013E01.npy  S018E01.npy  S023E01.npy  S028E01.npy
S003E02.npy  S008E02.npy  S013E02.npy  S018E02.npy  S023E02.npy  S028E02.npy
S003E03.npy  S008E03.npy  S013E03.npy  S018E03.npy  S023E03.npy  S028E03.npy
S003E04.npy  S008E04.npy  S013E04.npy  S018E04.npy  S023E04.npy  S028E04.npy
S004E01.npy  S009E01.npy  S014E01.npy  S019E01.npy  S024E01.npy  S029E01.npy

In [8]:
base_path = "."
concentration_data_array = None
relaxed_data_array = None
print("loading data from .npy files...")
for filename in os.listdir(base_path):
  if "E02" in filename and filename.endswith('.npy') or "E04" in filename and filename.endswith('.npy'):
    print(filename)
    concentration_data_array = np.concatenate((concentration_data_array, np.load(filename))) if concentration_data_array is not None else np.load(filename)
  if "E01" in filename and filename.endswith('.npy') or "E03" in filename and filename.endswith('.npy'):
    print(filename)
    relaxed_data_array = np.concatenate((relaxed_data_array, np.load(filename))) if relaxed_data_array is not None else np.load(filename)
print(concentration_data_array.shape, relaxed_data_array.shape)
print(len(concentration_data_array), len(relaxed_data_array))

loading data from .npy files...
S004E01.npy
S004E02.npy
S004E03.npy
S004E04.npy
S005E01.npy
S005E02.npy
S005E03.npy
S005E04.npy
S006E01.npy
S006E02.npy
S006E03.npy
S006E04.npy
S007E01.npy
S007E02.npy
S007E03.npy
S007E04.npy
S008E01.npy
S008E02.npy
S008E03.npy
S008E04.npy
S009E01.npy
S009E02.npy
S009E03.npy
S009E04.npy
S010E01.npy
S010E02.npy
S010E03.npy
S010E04.npy
S011E01.npy
S011E02.npy
S011E03.npy
S011E04.npy
S012E01.npy
S012E02.npy
S012E03.npy
S012E04.npy
S013E01.npy
S013E02.npy
S013E03.npy
S013E04.npy
S014E01.npy
S014E02.npy
S014E03.npy
S014E04.npy
S015E01.npy
S015E02.npy
S015E03.npy
S015E04.npy
S016E01.npy
S016E02.npy
S016E03.npy
S016E04.npy
S017E01.npy
S017E02.npy
S017E03.npy
S017E04.npy
S018E01.npy
S018E02.npy
S018E03.npy
S018E04.npy
S019E01.npy
S019E02.npy
S019E03.npy
S019E04.npy
S020E01.npy
S020E02.npy
S020E03.npy
S020E04.npy
S021E01.npy
S021E02.npy
S021E03.npy
S021E04.npy
S022E01.npy
S022E02.npy
S022E03.npy
S022E04.npy
S023E01.npy
S023E02.npy
S023E03.npy
S023E04.npy
S024E01.

In [9]:
X = np.concatenate((relaxed_data_array, concentration_data_array))
Y = np.concatenate((np.zeros((len(relaxed_data_array),1)),np.ones((len(concentration_data_array),1))))
print(X.shape, Y.shape)
relaxed_data_array, concentration_data_array = None, None

(4320, 14, 65, 45) (4320, 1)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)
X, Y = None, None
folds = list(StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(X_train, y_train))

In [0]:
model_name = "4-layer-Convolutions-raw"
def get_model():
  model = Sequential()
  
  model.add(Conv2D(64, kernel_size=5, strides=(1,1), padding='valid', activation='relu', input_shape = (14, 65, 45), data_format='channels_first')) #change the data format as the 14 channels are before the X,Y axes
  model.add(Conv2D(128, kernel_size=3, activation='relu'))
  model.add(MaxPooling2D(pool_size=(3, 3)))

  model.add(Conv2D(256, kernel_size=3, strides=(1,1), padding='valid', activation='relu'))
  model.add(Conv2D(512, kernel_size=2, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  model.add(Flatten())
  model.add(Dense(100, activation='relu'))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  
  sgd = optimizers.SGD(lr=0.000001, momentum=0.7, decay=1e-6)
  model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [13]:
filepath= model_name + "-weights-improvement-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model = get_model()
for j, (train_idx, val_idx) in enumerate(folds):
  print('\nFold ',j)
  X_train_cv = X_train[train_idx]
  y_train_cv = y_train[train_idx]
  X_valid_cv = X_train[val_idx]
  y_valid_cv= y_train[val_idx]
  # checkpoint
  model.fit(X_train_cv, y_train_cv, validation_data=(X_valid_cv, y_valid_cv), epochs=10, batch_size=32, shuffle=True)
score = model.evaluate(X_test,y_test)
print("Test set metrics: %s: %.2f%%" % (model.metrics_names[1], score[1]*100))
model_json = model.to_json()
with open(model_name + ".json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(model_name + ".h5")
print("Saved model %s to disk" % (model_name))


Fold  0
Train on 3304 samples, validate on 368 samples
Epoch 1/10
  32/3304 [..............................] - ETA: 1:12

InvalidArgumentError: ignored

In [0]:
# load json and create model
json_file = open(model_name+ '.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(model_name + ".h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
score = loaded_model.evaluate(X_test,y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))